In [ ]:
import astropy
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import os
from pathlib import Path
from glob import glob
import re
import logging
from rich.progress import Progress
import numpy as np
from sunpy.map import Map
import pandas as pd
from datetime import datetime, date, timedelta
import sscws
from sunpy.net import Fido, attrs as a
from functools import reduce

from matplotlib.colors import LogNorm, PowerNorm, Normalize
#from tqdm.notebook import tqdm
from tqdm import tqdm

import warnings

from sunpy.map.maputils import all_coordinates_from_map
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
base_path = '/mnt/onboard_data/visualization/cme_video'
os.makedirs(base_path, exist_ok=True)

In [ ]:
fnames_a = sorted(glob("/mnt/onboard_data/data/cor2/20140222_*_n*a.fts"))
fnames_b = sorted(glob("/mnt/onboard_data/data/cor2/20140222_*_n*b.fts"))

print(len(fnames_a), len(fnames_b))

In [ ]:
warnings.simplefilter('ignore')
for f in tqdm(fnames_a):
    # plot map to jpg
    m = Map(f)
    m = Map(np.log(m.data / m.exposure_time.value), m.meta)
    #print('VALUE RANGE', m.data.min(), m.data.max())
    #print(m.exposure_time)
    plt.figure(figsize=(4, 4))
    m.plot(norm=Normalize(vmin=4, vmax=7))
    plt.savefig(os.path.join(base_path, 'cor2sa_' + os.path.basename(f).replace('.fts', '.jpg')), dpi=100)
    plt.close()

In [ ]:
f = fnames_a[50]
print(f)

m = Map(f)

pixel_coords = all_coordinates_from_map(m)
solar_center = SkyCoord(0*u.deg, 0*u.deg, frame=m.coordinate_frame)
pixel_radii = np.sqrt((pixel_coords.Tx-solar_center.Tx)**2 +
                      (pixel_coords.Ty-solar_center.Ty)**2)
# r2 masking
mask = 1 - (pixel_radii / pixel_radii.max()) ** 2
mask = mask.value
mask[pixel_radii.value >= 0.9 * pixel_coords.Tx.max().value] = np.nan

m = Map(np.log(m.data / m.exposure_time.value / mask), m.meta)
m.plot(norm=Normalize())
plt.show()

print(m.data.min(), m.data.max())

In [ ]:
plt.imshow(mask)
plt.colorbar()
plt.show()

In [ ]:
m.data.min(), m.data.max(), m.exposure_time